In [1]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    PDFMinerLoader,
    PyMuPDFLoader,
    UnstructuredPDFLoader,
)


c:\Users\Bengulore Hemanth\Desktop\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
try:
    pypdf_loader =PyPDFLoader("data/pdf/attention.pdf")
    pyPyPDF_docs=pypdf_loader.load()
    print(pyPyPDF_docs)
except Exception as e :
    print(f"error:{e}")
print(len(pyPyPDF_docs[0].page_content))

incorrect startxref pointer(1)
parsing for Object Streams
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(2,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(202,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(321,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(1017,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(1646,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(1647,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(1648,0) whereas 100 expected
Error -3 while decompressing data: incorrect header check
found 0 objects within Object(1649,0) whereas 100 expected
Error -3 wh

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/pdf/attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content=''), Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/pdf/attention.pdf', 'total_pages': 15, 'page': 1, 'page_label': '2'}, page_content=''), Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperre

In [41]:
try:
    pymupdf_loader = PyMuPDFLoader("data/pdf/attention.pdf")
    pymupdf_docs = pymupdf_loader.load()

    print(f" Loaded {len(pymupdf_docs)} pages")
    print(f" Includes detailed metadata")
    print(pymupdf_docs)

except Exception as e:
    print(f" Error: {e}")

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: invalid code lengths set

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

 Loaded 15 pages
 Includ

In [2]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

In [3]:
import pytesseract
from langchain_community.document_loaders import (
    PyMuPDFLoader,
    PDFPlumberLoader,
    UnstructuredPDFLoader,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List


class SmartPDFCleaner:
    def __init__(self, chunk_size=1000, chunk_overlap=100):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=[" "]
        )

    def _clean_text(self, text: str) -> str:
        return " ".join(text.split())

    def process_pdf(self, pdf_path: str) -> List[Document]:
        """
        Auto strategy:
        1) Try PyMuPDF (digital PDFs)
        2) Fallback to PDFPlumber (broken text PDFs)
        3) Final fallback to OCR (scanned PDFs)
        """

        pages = []

        # 1️⃣ Try PyMuPDF
        try:
            loader = PyMuPDFLoader(pdf_path)
            pages = loader.load()
        except Exception:
            pages = []

        # 2️⃣ Fallback to PDFPlumber
        if not pages or all(len(p.page_content.strip()) <= 10 for p in pages):
            try:
                loader = PDFPlumberLoader(pdf_path)
                pages = loader.load()
            except Exception:
                pages = []

        # 3️⃣ Final fallback → OCR
        if not pages or all(len(p.page_content.strip()) <= 10 for p in pages):
            loader = UnstructuredPDFLoader(
                pdf_path,
                strategy="ocr_only",
                mode="elements"
            )
            pages = loader.load()

        processed_chunks = []

        for idx, page in enumerate(pages):
            cleaned_text = self._clean_text(page.page_content)

            # Skip very small OCR noise
            if len(cleaned_text.strip()) <= 10:
                continue

            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[{
                    **page.metadata,
                    "page": page.metadata.get("page_number", idx + 1),
                    "chunk_method": "smart_pdf_auto_loader",
                    "char_count": len(cleaned_text)
                }]
            )

            processed_chunks.extend(chunks)

        return processed_chunks


In [4]:
preprocessor=SmartPDFCleaner()

In [45]:
preprocessor

In [7]:
#process a PDF
preprocessor=SmartPDFCleaner()
try:
    chunks=preprocessor.process_pdf('data/pdf/test.pdf')
    print(len(chunks))
    print("Total chunks:", len(chunks))
    print(chunks[0].page_content[:300])

except Exception as e :
    print(e)

17
Total chunks: 17
1 The Oriental Insurance Co. Ltd STUDENT SAFETY POLICY UIN: IRDA/NL- HLT/OIC/P- H/V .1/22/14-15 Policy The Oriental Insurance Company Limited Head Office: A 25/27, Asaf Ali Road, New Delhi -110002 STUDENTS SAFETY POLICY ‘WHEREAS the insured named in the schedule herein (hereinafter called the ‘Insur


In [6]:
preprocessor = SmartPDFCleaner()
chunks = preprocessor.process_pdf("data/pdf/test.pdf")

print("Total chunks:", len(chunks))
print(chunks[0].page_content[:300])


Total chunks: 17
1 The Oriental Insurance Co. Ltd STUDENT SAFETY POLICY UIN: IRDA/NL- HLT/OIC/P- H/V .1/22/14-15 Policy The Oriental Insurance Company Limited Head Office: A 25/27, Asaf Ali Road, New Delhi -110002 STUDENTS SAFETY POLICY ‘WHEREAS the insured named in the schedule herein (hereinafter called the ‘Insur
